# EasyCV图像检测-ViTDet
本文将以[ViTDet](https://arxiv.org/abs/2203.16527)模型为例，介绍如何基于easyCV，和mmdet兼容进行目标检测模型训练和预测

## 运行环境要求

PAI-Pytorch镜像 or 原生Pytorch1.5+以上环境 GPU机器， 内存32G以上

## 安装依赖包

注: 在PAI-DSW docker中无需安装相关依赖，可跳过此部分 在本地notebook环境中执行


1、 首先，安装pytorch和对应版本的torchvision，支持Pytorch1.5.1以上版本

In [ ]:
# install pytorch and torch vision
! conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cudatoolkit=10.2 -c pytorch （根据cuda版本调整，详细可参考https://pytorch.org/get-started/previous-versions/）

2、 获取torch和cuda版本，安装对应版本的mmcv和nvidia-dali

In [ ]:
import torch
import os
os.environ['CUDA']='cu' + torch.version.cuda.replace('.', '')
os.environ['Torch']='torch'+torch.version.__version__.replace('+PAI', '')
!echo "cuda version: $CUDA"
!echo "pytorch version: $Torch"

In [ ]:
# install some python deps
! pip install mmcv-full -i https://pypi.tuna.tsinghua.edu.cn/simple （or pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/{cu_version}/{torch_version}/index.html  需要指定{}内的cuda版本和torch版本）
! pip install http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/third_party/nvidia_dali_cuda100-0.25.0-1535750-py3-none-manylinux2014_x86_64.whl

3、  安装EasyCV依赖包

In [ ]:
! pip install onnx -i https://pypi.tuna.tsinghua.edu.cn/simple 
! pip install mmdet -i https://pypi.tuna.tsinghua.edu.cn/simple 
! pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple 

4、 简单验证

In [ ]:
from easycv.apis import *

## 图像检测模型训练&预测

### 数据准备

你可以下载[COCO2017](https://cocodataset.org/#download)数据，也可以使用我们提供了示例COCO数据

In [ ]:
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/small_coco_demo/small_coco_demo.tar.gz && tar -zxf small_coco_demo.tar.gz

重命名数据文件，使其和COCO数据格式完全一致

In [ ]:
!mkdir -p data/  && mv small_coco_demo data/coco

data/coco格式如下

```shell
data/coco/
├── annotations
│   ├── instances_train2017.json
│   └── instances_val2017.json
├── train2017
│   ├── 000000005802.jpg
│   ├── 000000060623.jpg
│   ├── 000000086408.jpg
│   ├── 000000118113.jpg
│   ├── 000000184613.jpg
│   ├── 000000193271.jpg
│   ├── 000000222564.jpg
│       ...
│   └── 000000574769.jpg
└── val2017
    ├── 000000006818.jpg
    ├── 000000017627.jpg
    ├── 000000037777.jpg
    ├── 000000087038.jpg
    ├── 000000174482.jpg
    ├── 000000181666.jpg
    ├── 000000184791.jpg
    ├── 000000252219.jpg
         ...
    └── 000000522713.jpg
```

### 模型训练和推理

调用EasyCV的ViTDet模型进行训练

In [ ]:
# 单机8卡（单机8卡需要将update_interval设置成8）
! CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 python -m torch.distributed.launch --nproc_per_node=8 --master_port=29500 tools/train.py configs/detection/vitdet/vitdet_100e.py --work_dir easycv/vitdet --launcher pytorch --fp16
# 8机8卡
! cp  EasyCV/tools/launch.py ./ && cp EasyCV/tools/train.py ./ && python -m launch --nproc_per_node=8 train configs/detection/vitdet_dlc/vitdet_100e.py --work_dir easycv/vitdet_100e --launcher pytorch --fp16

调用EasyCV的ViTDet模型进行推理

In [ ]:
! CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 python -m torch.distributed.launch --nproc_per_node=8 --master_port=29500 tools/eval.py configs/detection/vitdet/vitdet_100e.py --work_dir easycv/vitdet --launcher pytorch --fp16 --eval

### 模型导出

In [ ]:
! python tools/export.py configs/detection/vitdet/vitdet_100e.py easycv/vitdet/epoch_100.pth  easycv/vitdet/vitdet_maskrcnn_export.pth

### 模型预测

In [ ]:
from easycv.predictors.detector import TorchViTDetPredictor

config_file = 'configs/detection/vitdet/vitdet_100e.py'
checkpoint_file = 'https://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/EasyCV/modelzoo/detection/vitdet/vit_base/vitdet_maskrcnn_export.pth'
img = 'https://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/demo/demo.jpg'
out_file = './result.jpg'
vitdet = TorchViTDetPredictor(config_file, checkpoint_file)
output = vitdet.predict(img)

vitdet.show_result_pyplot(img, output, show=True, out_file=out_file)